### Challenge 40: Implement an E=3 RSA Broadcast attack

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [2]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import cryptopals as cp

<div class="alert alert-block alert-info">

Assume you're a Javascript programmer. That is, you're using a naive handrolled RSA to encrypt without padding.

Assume you can be coerced into encrypting the same plaintext three times, under three different public keys. You can; it's happened.
</div>

Let's generate three different public/private key pairs.  Then encrypt a message under each and decrypt it using the actual private key to make sure everything is working properly.

In [34]:
key_size = 2048
[e0,d0,n0] = cp.genRSA_keypair(key_size)
[e1,d1,n1] = cp.genRSA_keypair(key_size)
[e2,d2,n2] = cp.genRSA_keypair(key_size)

message = int(b'Be careful!  Eve is watching!!'.hex(), 16)
#message = int(b'TEST'.hex(), 16)

c0 = pow(message, e0, n0)
c1 = pow(message, e1, n1)
c2 = pow(message, e2, n2)

# Make sure the key pairs are valid...
p0 = pow(c0, d0, n0)
p1 = pow(c1, d1, n1)
p2 = pow(c2, d2, n2)

print(hex(p0))
print(hex(p1))
print(hex(p2))
print()
print(bytes.fromhex(hex(p0)[2:]).decode())

0x4265206361726566756c212020457665206973207761746368696e672121
0x4265206361726566756c212020457665206973207761746368696e672121
0x4265206361726566756c212020457665206973207761746368696e672121

Be careful!  Eve is watching!!


<div class="alert alert-block alert-info">

Then an attacker can trivially decrypt your message, by:

1. Capturing any 3 of the ciphertexts and their corresponding pubkeys
2. Using the CRT to solve for the number represented by the three ciphertexts (which are residues mod their respective pubkeys)
3. Taking the cube root of the resulting number

The CRT says you can take any number and represent it as the combination of a series of residues mod a series of moduli. In the three-residue case, you have:
```python
result =
  (c_0 * m_s_0 * invmod(m_s_0, n_0)) +
  (c_1 * m_s_1 * invmod(m_s_1, n_1)) +
  (c_2 * m_s_2 * invmod(m_s_2, n_2)) mod N_012
```
where:

`c_0`, `c_1`, `c_2` are the three respective residues mod
 `n_0`, `n_1`, `n_2`

 `m_s_n` (for `n` in `0, 1, 2`) are the product of the moduli
 EXCEPT `n_n` --- ie, `m_s_1 = n_0 * n_2`

 `N_012` is the product of all three moduli
    
To decrypt RSA using a simple cube root, leave off the final modulus operation; just take the raw accumulated result and cube-root it.

</div>

<div class="alert alert-block alert-warning">

**Note**:  I wasn't able to get this to work when leaving off the final modulus...works perfectly with the final modulus.  
    
    TODO:  Figure out why.

</div>

In [35]:
ms0 = n1*n2
ms1 = n0*n2
ms2 = n0*n1

N012 = n0*n1*n2

result = ((c0 * ms0 * cp.invmod(ms0, n0)) + \
         (c1 * ms1 * cp.invmod(ms1, n1)) + \
         (c2 * ms2 * cp.invmod(ms2, n2))) % N012

To find the cube root of a very large integer, I use the implementation from Rosetta code:

In [36]:
def root(root, b):
    
    if b < 2:
        return b
    a1 = root - 1
    c = 1
    d = (a1 * c + b // (c ** a1)) // root
    e = (a1 * d + b // (d ** a1)) // root
    while c not in (d, e):
        c, d, e = d, e, (a1 * e + b // (e ** a1)) // root
    return min(d, e)

In [37]:
PT_n = hex(root(3, result))[2:]
print('Attempting to decrypt message:\n\n')
print(bytes.fromhex(PT_n).decode())

Attempting to decrypt message:


Be careful!  Eve is watching!!


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)